## Задача 1
В файле gsm_list.txt содержатся идентификаторы GEO NCBI (идентификаторы вида GSM#####). Вам необходимо подгрузить html-страницу с информацией по каждому эксперименту. Например, https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5176077. Обратите внимание на то, как устроена URL-ссылка. Ознакомтесь со страницей и имеющейся на ней информацией. Напишите программу, которая ищет соответствие серии экспериментов и эксперимента. Результат запищите ввиде таблицы, состоящей из 2 колонок.

\* В базе данных GEO NCBI некоторые из серий экспермиентов являются суперсериями, т. е. включают в себя другие серии экспериментов. Информацию об этом можно найти на соответствующих им страницам. Модифицируйте вашу программу так, чтобы идентификаторы суперсерий игнорировались

In [60]:
# Место для решения

import os
path_to_code = os.getcwd()
path_to_inp_file = os.path.join(path_to_code, os.pardir, 'lections', 'files_and_images', 'gsm_list.txt')

with open(path_to_inp_file, 'r') as inp_file:
    list_of_id = []
    for line in inp_file:
        line = line.rstrip()
        list_of_id.append(line)

import json
import requests
import bs4

import re
regexp_gse = re.compile(r'GSE\d+')
regexp_sup_ser = re.compile(r'This SuperSeries is composed of the SubSeries')


for elem_exp in list_of_id:
    response = requests.get(f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={elem_exp}')
    list_of_uniq_ser = list(set(regexp_gse.findall(response.text)))
    list_of_uniq_ser_wo_sup_ser = []
    for elem_ser in list_of_uniq_ser:
        response = requests.get(f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={elem_ser}')
        if regexp_sup_ser.search(response.text) == None:
            list_of_uniq_ser_wo_sup_ser.append(elem_ser)
    print(f'Exp: {elem_exp}| Series: {list_of_uniq_ser}\n')
    print(f'Exp: {elem_exp}| Series w/o SuperSer: {list_of_uniq_ser_wo_sup_ser}\n\n')
    


Exp: GSM2912912| Series: ['GSE108747']

Exp: GSM2912912| Series w/o SuperSer: ['GSE108747']


Exp: GSM4061022| Series: ['GSE136888', 'GSE136889']

Exp: GSM4061022| Series w/o SuperSer: ['GSE136888']


Exp: GSM3243597| Series: ['GSE116601', 'GSE116598']

Exp: GSM3243597| Series w/o SuperSer: ['GSE116598']


Exp: GSM2466565| Series: ['GSE78099']

Exp: GSM2466565| Series w/o SuperSer: ['GSE78099']


Exp: GSM772768| Series: ['GSE17312']

Exp: GSM772768| Series w/o SuperSer: ['GSE17312']


Exp: GSM4340006| Series: ['GSE145160']

Exp: GSM4340006| Series w/o SuperSer: ['GSE145160']


Exp: GSM3720853| Series: ['GSE129724']

Exp: GSM3720853| Series w/o SuperSer: ['GSE129724']


Exp: GSM4194777| Series: ['GSE141085']

Exp: GSM4194777| Series w/o SuperSer: ['GSE141085']


Exp: GSM2095176| Series: ['GSE79423']

Exp: GSM2095176| Series w/o SuperSer: ['GSE79423']


Exp: GSM3934871| Series: ['GSE134053']

Exp: GSM3934871| Series w/o SuperSer: ['GSE134053']


Exp: GSM5104304| Series: ['GSE167381']

Ex

## Задача 2
В файле encode_id_list.txt содержатся идентификаторы базы данных ENCODE (идентификаторы с префиксом ENCSR). Вам необходимо подгрузить json с информацией по каждому эксперименту. Например, https://www.encodeproject.org/experiments/ENCSR747VUU/?format=json. Обратите внимание на то, как устроена URL-ссылка. Ознакомтесь с JSON'ом и имеющейся в нём информацией. Напишите программу, которая собирает информацию об типе эксперимента (assay_title), таргете (target -> label) и дате сабмита данных (date_submitted) для каждого идентификатора. Результат представьте в виде таблицы, состоящей из 4 колонок.

In [93]:
# Место для решения

import os
path_to_code = os.getcwd()
path_to_inp_file = os.path.join(path_to_code, os.pardir, 'lections', 'files_and_images', 'encode_id_list.txt')

with open(path_to_inp_file, 'r') as inp_file:
    list_of_id = []
    for line in inp_file:
        line = line.rstrip()
        list_of_id.append(line)
        
import json
import requests
import bs4
from pprint import pprint as pp

import pandas as pd

data = []
for elem_id in list_of_id:
    response = requests.get(f'https://www.encodeproject.org/experiments/{elem_id}/?format=json')
    encode_json = json.loads(response.text)
    if encode_json.get('target') != None:
        data.append( [elem_id, encode_json['assay_title'], encode_json['target']['label'], encode_json['date_submitted']] )
    else:
        data.append( [elem_id, encode_json['assay_title'], None, encode_json['date_submitted']] )
result_table = pd.DataFrame(data, columns = ['ID', 'Type', 'Target', 'Date'])
result_table

,ID,Type,Target,Date
0,ENCSR668GKQ,Histone ChIP-seq,H3K9me3,2013-07-31
1,ENCSR495INQ,DNase-seq,None,2013-07-31
2,ENCSR518FUJ,TF ChIP-seq,pdm3,2015-07-24
3,ENCSR691IZA,Histone ChIP-seq,H3K36me3,2013-07-31
4,ENCSR316MNI,Histone ChIP-seq,H3K9me3,2013-07-31
5,ENCSR817QKV,TF ChIP-seq,EP400,2016-04-27
6,ENCSR642LNB,Control ChIP-seq,None,2013-07-31
7,ENCSR046FQX,CRISPRi RNA-seq,LDB1,2017-10-31
8,ENCSR394MHV,TF ChIP-seq,unpg,2018-09-18
9,ENCSR814XCZ,Control ChIP-seq,None,2017-10-13
